In [1]:
#notes
"""
how to install mysql jar
# https://dev.mysql.com/get/Downloads/Connector-J/mysql-connector-java_8.0.23-1ubuntu20.10_all.deb
download and run command
sudo dpkg -i mysql-connector-java_8.0.23-1ubuntu20.10_all.deb

after installation checj in cd /usr/share/java/mysql-connector-java-8.0.23.jar
"""

'\nhow to install mysql jar\n# https://dev.mysql.com/get/Downloads/Connector-J/mysql-connector-java_8.0.23-1ubuntu20.10_all.deb\ndownload and run command\nsudo dpkg -i mysql-connector-java_8.0.23-1ubuntu20.10_all.deb\n\nafter installation checj in cd /usr/share/java/mysql-connector-java-8.0.23.jar\n'

In [2]:
import findspark
findspark.init('/home/sparkTutorial/spark-2.4.7-bin-hadoop2.7')

In [3]:
import mysql.connector

In [4]:
#create spark session
import pyspark
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
import pyspark # only run after findspark.init()

spark = SparkSession \
        .builder \
        .appName('test') \
        .master('local[*]') \
        .config("spark.driver.extraClassPath", "/usr/share/java/mysql-connector-java-8.0.23.jar") \
        .getOrCreate()

spark


In [5]:
#create a class that extract load and transform data 
class ETL_mysql():
    def __init__(self,db_name,user,password):
        self.db_name = db_name
        self.user=user
        self.password=password
        
    #load data from mysql
    def load_table(self,table_name):
        table = spark.read.format("jdbc") \
                .option("url","jdbc:mysql://localhost:3306/%s" % (self.db_name)) \
                .option("driver","com.mysql.jdbc.Driver") \
                .option("dbtable",table_name) \
                .option("user",self.user) \
                .option("password",self.password).load()
        print("table %s load successfully" %(table_name))
        return table
    #perform join on three mysql table
    
    def join_with_sql(self):
        sql_query="""
                select employees.emp_no,employees.first_name,employees.last_name,
                departments.dept_name from employees inner join dept_emp on 
                employees.emp_no=dept_emp.emp_no inner join departments on 
                dept_emp.dept_no=departments.dept_no order by employees.emp_no asc
                """
        result=spark.sql(sql_query)
        return result
    
    def join_with_df(self,employees_df,departments_df,dept_emp_df):
        emp_dept_df = employees_df.join(dept_emp_df, ["emp_no"],"inner").select(['emp_no','first_name','last_name','gender','dept_no'])
        #now join emp_dept_df to department df to get department name associate with emp
        result=emp_dept_df.join(departments_df,['dept_no'],how='inner').select(['emp_no','first_name','last_name','dept_name'])
        #result.limit(4).show()
        return result


In [6]:
db_name='employees'
user='root'
password='Meta@123'
#create class object and pass some argument
obj=ETL_mysql(db_name,user,password)

In [7]:
#loading table mysql db
employees_df=obj.load_table('employees')
dept_emp_df=obj.load_table('dept_emp')
departments_df=obj.load_table('departments')
titles_df=obj.load_table('titles')


table employees load successfully
table dept_emp load successfully
table departments load successfully
table titles load successfully


In [8]:
print("emp_df")
dept_emp_df.limit(2).show()
print("department")
departments_df.limit(2).show()
print("employees_df")
employees_df.limit(2).show()


emp_df
+------+-------+----------+----------+
|emp_no|dept_no| from_date|   to_date|
+------+-------+----------+----------+
| 10001|   d005|1986-06-26|9999-01-01|
| 10002|   d007|1996-08-03|9999-01-01|
+------+-------+----------+----------+

department
+-------+----------------+
|dept_no|       dept_name|
+-------+----------------+
|   d009|Customer Service|
|   d005|     Development|
+-------+----------------+

employees_df
+------+----------+----------+---------+------+----------+
|emp_no|birth_date|first_name|last_name|gender| hire_date|
+------+----------+----------+---------+------+----------+
| 10001|1953-09-02|    Georgi|  Facello|     M|1986-06-26|
| 10002|1964-06-02|   Bezalel|   Simmel|     F|1985-11-21|
+------+----------+----------+---------+------+----------+



In [9]:
#join table with dataframe
result_df=obj.join_with_df(employees_df,departments_df,dept_emp_df)
result_df.show()

+------+-----------+---------+-----------+
|emp_no| first_name|last_name|  dept_name|
+------+-----------+---------+-----------+
| 10206|   Alassane|  Iwayama|Development|
| 10623| Aleksander|   Danlos|Development|
| 11033|    Shushma|     Bahk|Development|
| 11141|    Vasiliy|Kermarrec|Development|
| 11317|   Shigeaki| Hagimont|Development|
| 11748|     Lihong| Massonet|Development|
| 12027|    Zhanqiu| Vuskovic|Development|
| 12940|   Odinaldo|   Farrar|Development|
| 13285|      Uinam|Lienhardt|Development|
| 13840|      Remco|    Demke|Development|
| 14450|   Fumitaka|Prochazka|Development|
| 15619|      Conor|  Gyorkos|Development|
| 17389|      Taiji| Kemmerer|Development|
| 17679|     Khalid|  Terlouw|Development|
| 18051|Constantijn|Rosenbaum|Development|
| 18654|      Navid| Stenning|Development|
| 18944|     Sastry|   Sgarro|Development|
| 20497|      Yefim|    Asser|Development|
| 22346|   Jingling|Kragelund|Development|
| 23571|      Supot|   Cronau|Development|
+------+---

In [10]:
# #join table with sql query
# sql_query="""
#         select employees.emp_no,employees.first_name,employees.last_name,
#         departments.dept_name from employees inner join dept_emp on 
#         employees.emp_no=dept_emp.emp_no inner join departments on 
#         dept_emp.dept_no=departments.dept_no order by employees.emp_no asc
#         """
# result=spark.sql(sql_query)
# return result

In [25]:
#create sql view of this df
titles_df.createOrReplaceTempView("titles")
employees_df.createOrReplaceTempView("employees")
dept_emp_df.createOrReplaceTempView("dept_emp")
departments_df.createOrReplaceTempView("departments")

In [26]:
spark.sql("select * from employees limit 2").show()
spark.sql("select * from dept_emp limit 2").show()
spark.sql("select * from departments limit 2").show()

+------+----------+----------+---------+------+----------+
|emp_no|birth_date|first_name|last_name|gender| hire_date|
+------+----------+----------+---------+------+----------+
| 10001|1953-09-02|    Georgi|  Facello|     M|1986-06-26|
| 10002|1964-06-02|   Bezalel|   Simmel|     F|1985-11-21|
+------+----------+----------+---------+------+----------+

+------+-------+----------+----------+
|emp_no|dept_no| from_date|   to_date|
+------+-------+----------+----------+
| 10001|   d005|1986-06-26|9999-01-01|
| 10002|   d007|1996-08-03|9999-01-01|
+------+-------+----------+----------+

+-------+----------------+
|dept_no|       dept_name|
+-------+----------------+
|   d009|Customer Service|
|   d005|     Development|
+-------+----------------+



In [13]:
"""
#change data type of column
result_df.printSchema()
df2 = result_df.withColumn("emp_no",col("emp_no").cast(StringType())) \
    .withColumn("first_name",col("first_name").cast(StringType())) \
    .withColumn("last_name",col("last_name").cast(StringType())) \
    .withColumn("dept_name",col("dept_name").cast(StringType()))
df2.printSchema()

"""

'\n#change data type of column\nresult_df.printSchema()\ndf2 = result_df.withColumn("emp_no",col("emp_no").cast(StringType()))     .withColumn("first_name",col("first_name").cast(StringType()))     .withColumn("last_name",col("last_name").cast(StringType()))     .withColumn("dept_name",col("dept_name").cast(StringType()))\ndf2.printSchema()\n\n'

In [11]:
#first create mysql table if not exist
import mysql.connector
mydb = mysql.connector.connect(
  host="0.0.0.0",
  user=user,
  password=password,
  database=db_name
)

In [12]:
try:
    mycursor = mydb.cursor()
    mycursor.execute("CREATE TABLE result_emp (id INT AUTO_INCREMENT PRIMARY KEY,emp_no INT, first_name VARCHAR(255),last_name VARCHAR(255),dept_name VARCHAR(255))")
except Exception as e:
    print("table already exist")
    pass

In [29]:
#save result in mysql

In [15]:
result_df.write.format('jdbc').options(
    url='jdbc:mysql://localhost:3306/%s'%(db_name),
    driver='com.mysql.jdbc.Driver',
    dbtable='result_emp',
    user=user,
    password=password).mode('append').save()

In [13]:
#save result into csv file
result_df.write.mode("overwrite").csv('result_df.csv')

In [14]:
result_df.write.mode("overwrite").csv('result_df.json')